In [1]:
import random
import time
import os
from abc import ABC, abstractmethod
from typing import List, Optional, Dict, Any
import json
import groq
import pyaudio
import wave
import tempfile

In [2]:
class Move(ABC):
    """Abstract base class for all racing moves - demonstrates Abstraction"""
    
    def __init__(self, name: str, fuel_cost: int, tire_damage: int, description: str):
        self._name = name  
        self._fuel_cost = fuel_cost
        self._tire_damage = tire_damage
        self._description = description
    
    @property
    def name(self) -> str:
        return self._name
    
    @property
    def fuel_cost(self) -> int:
        return self._fuel_cost
    
    @property
    def tire_damage(self) -> int:
        return self._tire_damage
    
    @property
    def description(self) -> str:
        return self._description
    
    @abstractmethod
    def execute(self, attacker: 'Driver', defender: 'Driver') -> Dict[str, Any]:
        """Execute the move - must be implemented by subclasses"""
        pass

In [3]:
class OffensiveMove(Move):
    """Offensive racing move - demonstrates Inheritance"""
    
    def __init__(self, name: str, fuel_cost: int, tire_damage: int, description: str, damage_to_opponent: int):
        super().__init__(name, fuel_cost, tire_damage, description)
        self._damage_to_opponent = damage_to_opponent
    
    def execute(self, attacker: 'Driver', defender: 'Driver') -> Dict[str, Any]:
        """Execute offensive move - demonstrates Polymorphism"""
        attacker.use_fuel(self._fuel_cost)
        attacker.damage_tires(self._tire_damage)
        
        # Calculate actual damage (can be reduced by defensive moves)
        actual_damage = self._damage_to_opponent
        
        return {
            'type': 'offensive',
            'move': self._name,
            'damage_dealt': actual_damage,
            'attacker_fuel_used': self._fuel_cost,
            'attacker_tire_damage': self._tire_damage
        }

In [4]:
class DefensiveMove(Move):
    """Defensive racing move - demonstrates Inheritance"""
    
    def __init__(self, name: str, fuel_cost: int, tire_damage: int, description: str, damage_reduction: int):
        super().__init__(name, fuel_cost, tire_damage, description)
        self._damage_reduction = damage_reduction
    
    def execute(self, attacker: 'Driver', defender: 'Driver') -> Dict[str, Any]:
        """Execute defensive move - demonstrates Polymorphism"""
        defender.use_fuel(self._fuel_cost)
        defender.damage_tires(self._tire_damage)
        
        return {
            'type': 'defensive',
            'move': self._name,
            'damage_reduction': self._damage_reduction,
            'defender_fuel_used': self._fuel_cost,
            'defender_tire_damage': self._tire_damage
        }

In [5]:

class Driver(ABC):
    """Abstract base class for F1 drivers - demonstrates Abstraction and Encapsulation"""
    
    def __init__(self, name: str, initial_tire_health: int = 100, initial_fuel: int = 100):
        self._name = name  # Encapsulation
        self._tire_health = initial_tire_health
        self._fuel = initial_fuel
        self._offensive_moves: List[OffensiveMove] = []
        self._defensive_moves: List[DefensiveMove] = []
        self._initialize_moves()
    
    @abstractmethod
    def _initialize_moves(self):
        """Initialize driver-specific moves - must be implemented by subclasses"""
        pass
    
    # Encapsulation - controlled access to private data
    @property
    def name(self) -> str:
        return self._name
    
    @property
    def tire_health(self) -> int:
        return self._tire_health
    
    @property
    def fuel(self) -> int:
        return self._fuel
    
    @property
    def is_alive(self) -> bool:
        return self._tire_health > 0
    
    @property
    def offensive_moves(self) -> List[OffensiveMove]:
        return self._offensive_moves.copy()  # Return copy to prevent external modification
    
    @property
    def defensive_moves(self) -> List[DefensiveMove]:
        return self._defensive_moves.copy()
    
    def use_fuel(self, amount: int) -> bool:
        """Use fuel - returns False if not enough fuel"""
        if self._fuel >= amount:
            self._fuel -= amount
            return True
        return False
    
    def damage_tires(self, damage: int):
        """Apply tire damage"""
        self._tire_health = max(0, self._tire_health - damage)
    
    def can_use_defensive(self, move: DefensiveMove) -> bool:
        """Check if driver can use a defensive move"""
        return self._fuel >= move.fuel_cost
    
    def get_available_defensive_moves(self) -> List[DefensiveMove]:
        """Get defensive moves that can be used (fuel restriction)"""
        return [move for move in self._defensive_moves if self.can_use_defensive(move)]
    
    @abstractmethod
    def select_offensive_move(self, voice_controller=None) -> OffensiveMove:
        """Select an offensive move - must be implemented by subclasses"""
        pass
    
    @abstractmethod
    def select_defensive_move(self, voice_controller=None) -> Optional[DefensiveMove]:
        """Select a defensive move - must be implemented by subclasses"""
        pass

In [6]:

class Verstappen(Driver):
    """Max Verstappen driver class - demonstrates Inheritance and Polymorphism"""
    
    def _initialize_moves(self):
        """Initialize Verstappen's specific moves"""
        # Verstappen's Offensive Moves
        self._offensive_moves = [
            OffensiveMove("Aggressive Overtake", 15, 8, "Bold overtaking maneuver", 20),
            OffensiveMove("Late Braking", 12, 5, "Brake later into corners", 15),
            OffensiveMove("Slipstream Attack", 10, 3, "Use slipstream for speed", 12),
            OffensiveMove("Dive Bomb", 18, 10, "Risky inside overtake", 25)
        ]
        
        # Verstappen's Defensive Tactics
        self._defensive_moves = [
            DefensiveMove("Defensive Block", 8, 2, "Block the racing line", 8),
            DefensiveMove("Strategic Position", 6, 1, "Hold optimal track position", 5),
            DefensiveMove("Counter Defense", 10, 3, "Quick defensive reaction", 10),
            DefensiveMove("Track Limit", 12, 4, "Use full track width", 12)
        ]
    
    def select_offensive_move(self, voice_controller=None) -> OffensiveMove:
        """Verstappen's move selection - demonstrates Polymorphism"""
        if voice_controller:
            return voice_controller.get_voice_command(self._offensive_moves, f"{self._name} (Offensive)")
        
        print(f"\n{self._name}'s Offensive Moves:")
        for i, move in enumerate(self._offensive_moves, 1):
            print(f"{i}. {move.name} (Fuel: {move.fuel_cost}, Tire: {move.tire_damage}) - {move.description}")
        
        while True:
            try:
                choice = int(input(f"Select {self._name}'s offensive move (1-{len(self._offensive_moves)}): ")) - 1
                if 0 <= choice < len(self._offensive_moves):
                    return self._offensive_moves[choice]
                print("Invalid choice. Try again.")
            except ValueError:
                print("Please enter a number.")
    
    def select_defensive_move(self, voice_controller=None) -> Optional[DefensiveMove]:
        """Verstappen's defensive selection"""
        available_moves = self.get_available_defensive_moves()
        if not available_moves:
            print(f"{self._name} has no fuel for defensive moves!")
            return None
        
        if voice_controller:
            available_moves.append(None)  # Add "no defense" option
            return voice_controller.get_voice_command(available_moves, f"{self._name} (Defensive)")
        
        print(f"\n{self._name}'s Defensive Options:")
        for i, move in enumerate(available_moves, 1):
            print(f"{i}. {move.name} (Fuel: {move.fuel_cost}, Tire: {move.tire_damage}) - {move.description}")
        print(f"{len(available_moves) + 1}. No Defense")
        
        while True:
            try:
                choice = int(input(f"Select defense (1-{len(available_moves) + 1}): ")) - 1
                if choice == len(available_moves):
                    return None  # No defense
                if 0 <= choice < len(available_moves):
                    return available_moves[choice]
                print("Invalid choice. Try again.")
            except ValueError:
                print("Please enter a number.")


In [7]:

class Mostafa(Driver):
    """Hassan Mostafa driver class - demonstrates Inheritance and Polymorphism"""
    
    def _initialize_moves(self):
        """Initialize Mostafa's specific moves"""
        # Mostafa's Offensive Moves
        self._offensive_moves = [
            OffensiveMove("Push Hard", 14, 7, "Maximum attack mode", 18),
            OffensiveMove("Corner Cut", 11, 4, "Aggressive corner entry", 14),
            OffensiveMove("DRS Attack", 9, 2, "Use DRS for overtake", 10),
            OffensiveMove("Tire Strategy", 16, 6, "Aggressive tire usage", 22)
        ]
        
        # Mostafa's Defensive Tactics
        self._defensive_moves = [
            DefensiveMove("Defensive Weave", 7, 1, "Weave to break slipstream", 6),
            DefensiveMove("Box Box", 13, 5, "Strategic pit defense", 15),
            DefensiveMove("Track Defense", 9, 2, "Defend racing line", 8),
            DefensiveMove("Counter Move", 11, 3, "Quick counter defense", 11)
        ]
    
    def select_offensive_move(self, voice_controller=None) -> OffensiveMove:
        """Mostafa's move selection - demonstrates Polymorphism"""
        if voice_controller:
            return voice_controller.get_voice_command(self._offensive_moves, f"{self._name} (Offensive)")
        
        print(f"\n{self._name}'s Offensive Moves:")
        for i, move in enumerate(self._offensive_moves, 1):
            print(f"{i}. {move.name} (Fuel: {move.fuel_cost}, Tire: {move.tire_damage}) - {move.description}")
        
        while True:
            try:
                choice = int(input(f"Select {self._name}'s offensive move (1-{len(self._offensive_moves)}): ")) - 1
                if 0 <= choice < len(self._offensive_moves):
                    return self._offensive_moves[choice]
                print("Invalid choice. Try again.")
            except ValueError:
                print("Please enter a number.")
    
    def select_defensive_move(self, voice_controller=None) -> Optional[DefensiveMove]:
        """Mostafa's defensive selection"""
        available_moves = self.get_available_defensive_moves()
        if not available_moves:
            print(f"{self._name} has no fuel for defensive moves!")
            return None
        
        if voice_controller:
            available_moves.append(None)  # Add "no defense" option
            return voice_controller.get_voice_command(available_moves, f"{self._name} (Defensive)")
        
        print(f"\n{self._name}'s Defensive Options:")
        for i, move in enumerate(available_moves, 1):
            print(f"{i}. {move.name} (Fuel: {move.fuel_cost}, Tire: {move.tire_damage}) - {move.description}")
        print(f"{len(available_moves) + 1}. No Defense")
        
        while True:
            try:
                choice = int(input(f"Select defense (1-{len(available_moves) + 1}): ")) - 1
                if choice == len(available_moves):
                    return None  # No defense
                if 0 <= choice < len(available_moves):
                    return available_moves[choice]
                print("Invalid choice. Try again.")
            except ValueError:
                print("Please enter a number.")


In [8]:
class VoiceController:
    """Voice control system using Groq API - Bonus feature"""
    
    def __init__(self, api_key: str):
        
        
        self.client = groq.Groq(api_key=api_key)
        self.audio_format = pyaudio.paInt16
        self.channels = 1
        self.rate = 44100
        self.chunk = 1024
        self.record_seconds = 3
        
        # Command mappings for voice recognition
        self.command_mappings = {
            # Verstappen commands
            "aggressive overtake": "Aggressive Overtake",
            "late braking": "Late Braking", 
            "slipstream attack": "Slipstream Attack",
            "dive bomb": "Dive Bomb",
            "defensive block": "Defensive Block",
            "strategic position": "Strategic Position",
            "counter defense": "Counter Defense",
            "track limit": "Track Limit",
            
            # Mostafa commands
            "push hard": "Push Hard",
            "corner cut": "Corner Cut",
            "drs attack": "DRS Attack",
            "tire strategy": "Tire Strategy",
            "defensive weave": "Defensive Weave",
            "box box": "Box Box",
            "track defense": "Track Defense",
            "counter move": "Counter Move",
            
            # Common commands
            "no defense": None,
            "skip": None,
            "pass": None
        }
    
    def record_audio(self) -> str:
        """Record audio from microphone and save to temporary file"""
        audio = pyaudio.PyAudio()
        
        # Create temporary file
        temp_file = tempfile.NamedTemporaryFile(delete=False, suffix=".wav")
        temp_filename = temp_file.name
        temp_file.close()
        
        print("🎤 Recording... Speak your command now!")
        
        stream = audio.open(
            format=self.audio_format,
            channels=self.channels,
            rate=self.rate,
            input=True,
            frames_per_buffer=self.chunk
        )
        
        frames = []
        for _ in range(0, int(self.rate / self.chunk * self.record_seconds)):
            data = stream.read(self.chunk)
            frames.append(data)
        
        stream.stop_stream()
        stream.close()
        audio.terminate()
        
        # Save audio to file
        wave_file = wave.open(temp_filename, 'wb')
        wave_file.setnchannels(self.channels)
        wave_file.setsampwidth(audio.get_sample_size(self.audio_format))
        wave_file.setframerate(self.rate)
        wave_file.writeframes(b''.join(frames))
        wave_file.close()
        
        print("🔄 Processing audio...")
        return temp_filename
    
    def transcribe_audio(self, audio_file_path: str) -> str:
        """Transcribe audio using Groq API"""
        try:
            with open(audio_file_path, "rb") as file:
                transcription = self.client.audio.transcriptions.create(
                    file=(audio_file_path, file.read()),
                    model="whisper-large-v3",
                    response_format="text"
                )
            
            # Clean up temporary file
            os.unlink(audio_file_path)
            
            return transcription.strip().lower()
        except Exception as e:
            print(f"❌ Transcription error: {e}")
            os.unlink(audio_file_path)
            return ""
    
    def match_command(self, transcription: str, available_moves: List) -> Optional[Any]:
        """Match transcription to available moves"""
        # Clean transcription
        transcription = transcription.lower().strip()
        
        # Direct mapping check
        if transcription in self.command_mappings:
            target_name = self.command_mappings[transcription]
            if target_name is None:
                return None  # "no defense" command
            
            # Find matching move
            for move in available_moves:
                if move and move.name == target_name:
                    return move
        
        # Fuzzy matching for similar words
        for move in available_moves:
            if move and any(word in transcription for word in move.name.lower().split()):
                return move
        
        return None
    
    def get_voice_command(self, available_moves: List, context: str) -> Any:
        """Get voice command from user"""
        print(f"\n🎮 Voice Control Active for {context}")
        print("Available commands:")
        for move in available_moves:
            if move:
                print(f"  - \"{move.name}\"")
        if None in available_moves or context.endswith("(Defensive)"):
            print("  - \"No Defense\" / \"Skip\" / \"Pass\"")
        
        max_attempts = 3
        attempts = 0
        
        while attempts < max_attempts:
            attempts += 1
            
            try:
                # Record audio
                audio_file = self.record_audio()
                
                # Transcribe
                transcription = self.transcribe_audio(audio_file)
                print(f"📝 Heard: \"{transcription}\"")
                
                if not transcription:
                    print("❌ No speech detected. Try again.")
                    continue
                
                # Match command
                matched_move = self.match_command(transcription, available_moves)
                
                if matched_move is not None or transcription in ["no defense", "skip", "pass"]:
                    if matched_move:
                        print(f"✅ Command recognized: {matched_move.name}")
                    else:
                        print("✅ No defense selected")
                    return matched_move
                else:
                    print(f"❌ Command not recognized. Try again ({max_attempts - attempts} attempts left)")
            
            except KeyboardInterrupt:
                print("\n⏭️ Voice control interrupted. Falling back to text input.")
                break
            except Exception as e:
                print(f"❌ Voice control error: {e}")
                break
        
        # Fallback to text input
        print("🔄 Falling back to keyboard input...")
        if context.endswith("(Offensive)"):
            return self._text_fallback_offensive(available_moves)
        else:
            return self._text_fallback_defensive(available_moves)
    
    def _text_fallback_offensive(self, moves: List[OffensiveMove]) -> OffensiveMove:
        """Text fallback for offensive moves"""
        print("Select move:")
        for i, move in enumerate(moves, 1):
            print(f"{i}. {move.name}")
        
        while True:
            try:
                choice = int(input("Enter choice: ")) - 1
                if 0 <= choice < len(moves):
                    return moves[choice]
            except ValueError:
                pass
            print("Invalid choice. Try again.")
    
    def _text_fallback_defensive(self, moves: List) -> Optional[DefensiveMove]:
        """Text fallback for defensive moves"""
        actual_moves = [m for m in moves if m is not None]
        print("Select defense:")
        for i, move in enumerate(actual_moves, 1):
            print(f"{i}. {move.name}")
        print(f"{len(actual_moves) + 1}. No Defense")
        
        while True:
            try:
                choice = int(input("Enter choice: ")) - 1
                if choice == len(actual_moves):
                    return None
                if 0 <= choice < len(actual_moves):
                    return actual_moves[choice]
            except ValueError:
                pass
            print("Invalid choice. Try again.")


In [9]:

class RaceSimulator:
    """Main race simulation class - demonstrates composition and orchestration"""
    
    def __init__(self, driver1: Driver, driver2: Driver, voice_controller: Optional[VoiceController] = None):
        self.driver1 = driver1
        self.driver2 = driver2
        self.current_turn = 0
        self.race_log = []
        self.voice_controller = voice_controller
    
    def clear_screen(self):
        """Clear console screen"""
        os.system('cls' if os.name == 'nt' else 'clear')
    
    def display_status(self):
        """Display current race status"""
        print("=" * 60)
        print("🏎️  SILVERSTONE F1 RACE - VERSTAPPEN vs MOSTAFA  🏎️")
        print("=" * 60)
        print(f"Turn: {self.current_turn}")
        print(f"{self.driver1.name}: Tire Health: {self.driver1.tire_health}% | Fuel: {self.driver1.fuel}%")
        print(f"{self.driver2.name}: Tire Health: {self.driver2.tire_health}% | Fuel: {self.driver2.fuel}%")
        print("-" * 60)
    
    def execute_turn(self, attacker: Driver, defender: Driver):
        """Execute a single turn of the race"""
        self.current_turn += 1
        
        print(f"\n🚦 Turn {self.current_turn}: {attacker.name}'s Attack Phase")
        
        # Attacker selects offensive move
        offensive_move = attacker.select_offensive_move(self.voice_controller)
        
        print(f"\n⚡ {attacker.name} uses: {offensive_move.name}")
        print(f"   {offensive_move.description}")
        
        # Defender selects defensive move
        print(f"\n🛡️  {defender.name}'s Defense Phase")
        defensive_move = defender.select_defensive_move(self.voice_controller)
        
        # Execute moves
        attack_result = offensive_move.execute(attacker, defender)
        defense_result = None
        
        if defensive_move:
            print(f"\n🛡️  {defender.name} responds with: {defensive_move.name}")
            print(f"   {defensive_move.description}")
            defense_result = defensive_move.execute(attacker, defender)
        else:
            print(f"\n💨 {defender.name} chooses no defense!")
        
        # Calculate final damage
        final_damage = attack_result['damage_dealt']
        if defense_result:
            final_damage = max(0, final_damage - defense_result['damage_reduction'])
        
        # Apply damage to defender
        defender.damage_tires(final_damage)
        
        # Log the turn
        turn_log = {
            'turn': self.current_turn,
            'attacker': attacker.name,
            'defender': defender.name,
            'offensive_move': offensive_move.name,
            'defensive_move': defensive_move.name if defensive_move else "None",
            'final_damage': final_damage,
            'attacker_status': {'tire': attacker.tire_health, 'fuel': attacker.fuel},
            'defender_status': {'tire': defender.tire_health, 'fuel': defender.fuel}
        }
        self.race_log.append(turn_log)
        
        # Display turn results
        print(f"\n📊 Turn Results:")
        print(f"   💥 Damage dealt to {defender.name}: {final_damage}")
        if defense_result:
            print(f"   🛡️  Damage reduced by: {defense_result['damage_reduction']}")
        
        time.sleep(1)  # Dramatic pause
    
    def check_winner(self) -> Optional[Driver]:
        """Check if there's a winner"""
        if not self.driver1.is_alive:
            return self.driver2
        elif not self.driver2.is_alive:
            return self.driver1
        return None
    
    def display_final_stats(self, winner: Driver):
        """Display final race statistics"""
        self.clear_screen()
        print("🏁" * 20)
        print(f"🏆 RACE WINNER: {winner.name}! 🏆")
        print("🏁" * 20)
        
        print(f"\n📈 Final Statistics:")
        print(f"{self.driver1.name}: Tire Health: {self.driver1.tire_health}% | Fuel: {self.driver1.fuel}%")
        print(f"{self.driver2.name}: Tire Health: {self.driver2.tire_health}% | Fuel: {self.driver2.fuel}%")
        
        print(f"\n🔄 Total Turns: {self.current_turn}")
        
        # Display race summary
        print(f"\n📝 Race Summary:")
        for log in self.race_log[-3:]:  # Show last 3 turns
            print(f"Turn {log['turn']}: {log['attacker']} → {log['offensive_move']} vs {log['defender']} → {log['defensive_move']}")
    
    def run_race(self):
        """Main race loop"""
        self.clear_screen()
        print("🏎️  Welcome to the F1 Silverstone Showdown!")
        print("🥊 Verstappen vs Mostafa - May the fastest driver win!")
        
        if self.voice_controller:
            print("🎤 Voice control enabled! Speak your racing commands!")
        
        input("\nPress Enter to start the race...")
        
        drivers = [self.driver1, self.driver2]
        attacker_index = 0
        
        while self.driver1.is_alive and self.driver2.is_alive:
            self.clear_screen()
            self.display_status()
            
            # Alternate between drivers
            attacker = drivers[attacker_index]
            defender = drivers[1 - attacker_index]
            
            self.execute_turn(attacker, defender)
            
            # Check for winner
            winner = self.check_winner()
            if winner:
                self.display_final_stats(winner)
                break
            
            # Switch turns
            attacker_index = 1 - attacker_index
            
            input("\nPress Enter for next turn...")
        
        print("\n🏁 Thanks for watching the F1 Silverstone Showdown!")


In [ ]:
def main():
    """Main function to run the F1 simulation"""
    print("🏎️  F1 Racing Simulation - Verstappen vs Mostafa")
    print("=" * 50)
    
    # Initialize drivers
    verstappen = Verstappen("Max Verstappen")
    mostafa = Mostafa("Hassan Mostafa")
    
    # Voice control setup
    voice_controller = None
    
    use_voice = input("🎤 Enable voice control? (y/n): ").lower().strip() == 'y'
    if use_voice:
            api_key = "YOUR_GROQ_API_KEY"
            voice_controller = VoiceController(api_key)
            print(" Voice control enabled!")
    else:
            print(" Voice control disabled. Using keyboard input.")
    simulator = RaceSimulator(verstappen, mostafa, voice_controller)
    simulator.run_race()

if __name__ == "__main__":
    main()

🏎️  F1 Racing Simulation - Verstappen vs Mostafa
 Voice control enabled!
🏎️  Welcome to the F1 Silverstone Showdown!
🥊 Verstappen vs Mostafa - May the fastest driver win!
🎤 Voice control enabled! Speak your racing commands!
🏎️  SILVERSTONE F1 RACE - VERSTAPPEN vs MOSTAFA  🏎️
Turn: 0
Max Verstappen: Tire Health: 100% | Fuel: 100%
Hassan Mostafa: Tire Health: 100% | Fuel: 100%
------------------------------------------------------------

🚦 Turn 1: Max Verstappen's Attack Phase

🎮 Voice Control Active for Max Verstappen (Offensive)
Available commands:
  - "Aggressive Overtake"
  - "Late Braking"
  - "Slipstream Attack"
  - "Dive Bomb"
🎤 Recording... Speak your command now!
🔄 Processing audio...
📝 Heard: "gracias."
❌ Command not recognized. Try again (2 attempts left)
🎤 Recording... Speak your command now!
🔄 Processing audio...
📝 Heard: "thank you."
❌ Command not recognized. Try again (1 attempts left)
🎤 Recording... Speak your command now!
🔄 Processing audio...
📝 Heard: "thank you."
❌ Com